In [1]:
import numpy as np
from sim.ebm.intervention import compose_intv
from sim.ebm.obj import load_obj_age
import pandas as pd
from tqdm import tqdm
import json

folder = 'out/post_dyage'

obj = load_obj_age(
    folder_input=f'pars',
    file_prior='data/prior.txt',
    file_targets='data/targets.csv',
    year0=2000
)

In [11]:
post = pd.read_csv('out/post_dyage/Post.csv')
post = [dict(row) for i, row in post.iterrows()]

mss = list()


pars = post[0]
p = obj.serve(pars)
p = obj.Cas.prepare_pars(p)


In [16]:
incs = list()

for pars in post[:10]:
    p = obj.serve(pars)
    p = obj.Cas.prepare_pars(p)
    _, ms, _ = obj.Model.simulate_to_fit(p, np.linspace(2022, 2023, 2))
    incs.append(ms.IncR.iloc[0])
    
incs = np.array(incs)

In [81]:
def mod(p, ks, obj):
    k_beta, k_rel = ks
    
    p = dict(p)
    p['beta'] = k_beta * p['beta']
    p['r_relapse_te'] = k_rel * p['r_relapse_te']
    
    p = obj.serve(p)
    p = obj.Cas.prepare_pars(p)
    
    _, ms, _ = obj.Model.simulate_to_fit(p, np.linspace(2022, 2023, 2))
    m = ms.iloc[0, :]
    
    r_rel_te = p['r_relapse_te']
    rr_rel_pub = p['rr_relapse_pub'] / (0.3 + 0.7 * p['rr_relapse_pub'])
    rr_rel_pri = 1 / (0.3 + 0.7 * p['rr_relapse_pub'])
    r_rel_teu, r_rel_tei = r_rel_te * rr_rel_pub, r_rel_te * rr_rel_pri
    
    return {
        'IncR': m.IncR,
        'PrTreated': m.IncTreatedR / m.IncR,
        'PrPubTrated': m.IncTreatedPubR / m.IncR,
        'PrRelPub': 1 - np.exp(- r_rel_teu),
        'PrRelPri': 1 - np.exp(- r_rel_tei)
    }

In [82]:
def mod2(p, ks, obj):
    k_beta, k_rel = ks
    
    p = dict(p)
    p['beta'] = k_beta * p['beta']
    p['r_relapse_te'] = k_rel * p['r_relapse_te']
    p['rr_relapse_pub'] = 1
    
    p = obj.serve(p)
    p = obj.Cas.prepare_pars(p)
    
    _, ms, _ = obj.Model.simulate_to_fit(p, np.linspace(2022, 2023, 2))
    m = ms.iloc[0, :]
    
    r_rel_te = p['r_relapse_te']
    rr_rel_pub = p['rr_relapse_pub'] / (0.3 + 0.7 * p['rr_relapse_pub'])
    rr_rel_pri = 1 / (0.3 + 0.7 * p['rr_relapse_pub'])
    r_rel_teu, r_rel_tei = r_rel_te * rr_rel_pub, r_rel_te * rr_rel_pri
    
    return {
        'IncR': m.IncR,
        'PrTreated': m.IncTreatedR / m.IncR,
        'PrPubTrated': m.IncTreatedPubR / m.IncR,
        'PrRelPub': 1 - np.exp(- r_rel_teu),
        'PrRelPri': 1 - np.exp(- r_rel_tei)
    }

In [95]:
incs0 = list()

for pars in post[:10]:
    incs0.append(mod(pars, [1, 1], obj))
    
incs0 = pd.DataFrame(incs0)
incs0.mean()

IncR           0.002047
PrTreated      0.087212
PrPubTrated    0.071329
PrRelPub       0.091657
PrRelPri       0.034209
dtype: float64

- In Prev: 18%
- Pub in Prev: 14%

In [90]:
incs1 = list()

for pars in post[:10]:
    incs1.append(mod(pars, [0.85, 3.2], obj))
    
incs1 = pd.DataFrame(incs1)
incs1.mean()

IncR           0.002090
PrTreated      0.192883
PrPubTrated    0.154651
PrRelPub       0.264801
PrRelPri       0.105345
dtype: float64

- Pub in Prev: 14%

In [98]:
incs2 = list()

for pars in post[:10]:
    incs2.append(mod2(pars, [0.77, 4.4], obj))
    
incs2 = pd.DataFrame(incs2)
incs2.mean()

IncR           0.002090
PrTreated      0.241393
PrPubTrated    0.159696
PrRelPub       0.289692
PrRelPri       0.289692
dtype: float64

- PreTreated in CNR: 10%

In [103]:
incs3 = list()

for pars in post[:10]:
    incs3.append(mod2(pars, [0.97, 1.2], obj))
    
incs3 = pd.DataFrame(incs3)
incs3.mean()

IncR           0.001952
PrTreated      0.106125
PrPubTrated    0.069663
PrRelPub       0.089069
PrRelPri       0.089069
dtype: float64